In [209]:
import networkx as nx
import pandas as pd
import networkx.algorithms.community as nx_comm
import community as community_louvain
import matplotlib.cm as cm
from statsmodels.distributions.empirical_distribution import ECDF
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import powerlaw


In [210]:
cities = pd.read_csv('/Users/vladislavluksha/cities.csv')

In [211]:
cities = cities.drop_duplicates()
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 19940
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   City_origin         624 non-null    object 
 1   Country_origin      624 non-null    object 
 2   Latitude_origin     624 non-null    float64
 3   Longitude_origin    624 non-null    float64
 4   departure_iataCode  624 non-null    object 
dtypes: float64(2), object(3)
memory usage: 29.2+ KB


In [175]:
import qwikidata
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out

In [176]:
def get_city_population(city, country):
    try:
        result = get_city_wikidata(city, country)
        population = int(result['population']['value'])
        return population
    except:
        return None

cities['population'] = cities.apply(lambda row: get_city_population(row['City_origin'], row['Country_origin']), axis=1)


In [177]:
cities.head

<bound method NDFrame.head of       City_origin Country_origin  Latitude_origin  Longitude_origin iso_code  \
0         Aalborg        Denmark        57.092759          9.849243      NaN   
1          Aarhus        Denmark        56.299999         10.619000      NaN   
2        Alicante          Spain        38.282200         -0.558156      NaN   
3         Antwerp        Belgium        51.189400          4.460280      NaN   
4       Stockholm         Sweden        59.651901         17.918600      NaN   
...           ...            ...              ...               ...      ...   
19935      Kardla        Estonia        58.990799         22.830700      NaN   
19936  Kuressaare        Estonia        58.229900         22.509501      NaN   
19938        Agen         France        44.174702          0.590556      NaN   
19939     Orleans         France        47.987801          1.760560      NaN   
19940    Braganca       Portugal        41.857800         -6.707130      NaN   

       po

In [212]:
import wbdata
import pandas as pd
import datetime

In [213]:
import pycountry

# create a function to get the ISO-3 code for a given country name
def get_iso_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_3
    except AttributeError:
        return None



In [214]:
# add a new column to the dataframe with the ISO-3 code for each country
cities['iso_code'] = cities['Country_origin'].apply(get_iso_code)


In [215]:
cities[cities['Country_origin'] == 'Macedonia']

,City_origin,Country_origin,Latitude_origin,Longitude_origin,departure_iataCode,iso_code
637,Skopje,Macedonia,41.961601,21.621401,SKP,None
19793,Ohrid,Macedonia,41.180000,20.742300,OHD,None


In [216]:
cities.loc[cities['Country_origin'] == 'Russia', 'iso_code'] = 'RUS'

In [217]:
cities.loc[cities['Country_origin'] == 'Czech Republic', 'iso_code'] = 'CZE'

In [218]:
cities.loc[cities['Country_origin'] == 'Moldova', 'iso_code'] = 'MDA'

In [219]:
cities.loc[cities['Country_origin'] == 'Macedonia', 'iso_code'] = 'MKD'

In [220]:
cities.head()

,City_origin,Country_origin,Latitude_origin,Longitude_origin,departure_iataCode,iso_code
0,Aalborg,Denmark,57.092759,9.849243,AAL,DNK
1,Aarhus,Denmark,56.299999,10.619000,AAR,DNK
2,Alicante,Spain,38.282200,-0.558156,ALC,ESP
3,Antwerp,Belgium,51.189400,4.460280,ANR,BEL
4,Stockholm,Sweden,59.651901,17.918600,ARN,SWE


In [221]:
import requests

def get_gdp(country_code, city_name):
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.CD?format=json"
    response = requests.get(url)
    data = response.json()
    
    # Extract GDP value for the most recent year available
    try:
        gdp = [d['value'] for d in data[1] if d['countryiso3code'] == country_code and d['date'] == '2020'][0]
        return gdp
    except:
        print(f"No GDP data found for {city_name}, {country_code}")
        return None


In [222]:
cities['GDP'] = cities.apply(lambda x: get_gdp(x['iso_code'], x['City_origin']), axis=1)


No GDP data found for Jersey, JEY
No GDP data found for Guernsey, GGY
No GDP data found for Alderney, GGY


In [223]:
import pycountry

def get_country_code(country_name):
    try:
        country_code = pycountry.countries.search_fuzzy(country_name)[0].alpha_2
    except LookupError:
        country_code = None
    return country_code

In [224]:
cities['two_letter_code'] = cities['Country_origin'].apply(get_country_code)

In [225]:
cities.at[337,'City_origin'] = 'Minsk'

In [194]:
import requests
import pandas as pd

# function to look up population based on country and city name using Geonames API
def get_population(city, country):
    username = 'vladluksha'
    url = f'http://api.geonames.org/searchJSON?q={city}&country={country}&maxRows=1&username={username}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get('geonames') and data.get('geonames')[0].get('population'):
            return data.get('geonames')[0].get('population')
    return None



In [195]:
cities['population'] = cities.apply(lambda x: get_population(x['City_origin'], x['iso_code']), axis=1)

In [196]:
cities.head()

,City_origin,Country_origin,Latitude_origin,Longitude_origin,iso_code,population,GDP,two_letter_code
0,Aalborg,Denmark,57.092759,9.849243,DNK,None,3.552224e+11,DK
1,Aarhus,Denmark,56.299999,10.619000,DNK,None,3.552224e+11,DK
2,Alicante,Spain,38.282200,-0.558156,ESP,None,1.276963e+12,ES
3,Antwerp,Belgium,51.189400,4.460280,BEL,None,5.252118e+11,BE
4,Stockholm,Sweden,59.651901,17.918600,SWE,None,5.470542e+11,SE


In [226]:
geonames = pd.read_csv('/Users/vladislavluksha/Downloads/geonames-all-cities-with-a-population-1000.csv', sep =';')

In [227]:
geonames = geonames[['Name','ASCII Name','Country Code','Population']]
geonames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140867 entries, 0 to 140866
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Name          140867 non-null  object
 1   ASCII Name    140866 non-null  object
 2   Country Code  140822 non-null  object
 3   Population    140867 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 4.3+ MB


In [228]:
merged_df = pd.merge(geonames, cities, how='right', left_on=['Name','Country Code'],right_on=['City_origin','two_letter_code'])



In [230]:
merged_df = merged_df.drop(columns=['Name','ASCII Name','Country Code'])

In [231]:
merged_df.head()

,Population,City_origin,Country_origin,Latitude_origin,Longitude_origin,departure_iataCode,iso_code,GDP,two_letter_code
0,142937.0,Aalborg,Denmark,57.092759,9.849243,AAL,DNK,3.552224e+11,DK
1,NaN,Aarhus,Denmark,56.299999,10.619000,AAR,DNK,3.552224e+11,DK
2,334757.0,Alicante,Spain,38.282200,-0.558156,ALC,ESP,1.276963e+12,ES
3,NaN,Antwerp,Belgium,51.189400,4.460280,ANR,BEL,5.252118e+11,BE
4,1515017.0,Stockholm,Sweden,59.651901,17.918600,ARN,SWE,5.470542e+11,SE


In [133]:
merged_df[~merged_df['Population'].isna()]

,Population,City_origin,Country_origin,GDP,iso_code,two_letter_code
0,142937.0,Aalborg,Denmark,3.552224e+11,DNK,DK
2,334757.0,Alicante,Spain,1.276963e+12,ESP,ES
4,1515017.0,Stockholm,Sweden,5.470542e+11,SWE,SE
5,1620343.0,Barcelona,Spain,1.276963e+12,ESP,ES
6,1273651.0,Belgrade,Serbia,5.335648e+10,SRB,RS
...,...,...,...,...,...,...
611,49232.0,Annecy,France,2.639009e+12,FRA,FR
612,28334.0,Geilenkirchen,Germany,3.889669e+12,DEU,DE
613,68082.0,Neubrandenburg,Germany,3.889669e+12,DEU,DE
614,64754.0,Nakhchivan,Azerbaijan,4.269300e+10,AZE,AZ


In [232]:
merged_df.to_csv('populationgdp.csv', index=False)

##### 